In [1]:
## ######################################### ##
## Link between Markit RED and CRSP          ##
## Relies on 6-digit CUSIP  & Ticker         ##      
## Author: Qingyi (Freda) Song Drechsler     ##      
## Date: December 2022                       ## 
## ######################################### ##

import wrds
import os
import numpy as np
import pandas as pd

# display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import pickle as pkl
from fuzzywuzzy import fuzz

###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Loading library list...
Done


In [2]:


### Get red entity information
redent = conn.get_table(library='markit', table='redent')

# Quick check to confirm that it is the header information
# i.e. each redcode is mapped to only one entity 
# and doesn't contain historical records
redcnt = redent.groupby(['redcode'])['entity_cusip'].count().reset_index().rename(columns={'entity_cusip':'cusipCnt'})
redcnt.cusipCnt.max()
### Each redcode has only one entity_cusip mapped
### so it is indeed only header record

### Get information from Mergent header table
mergentHdr = conn.raw_sql('select permno, permco, hdrcusip, ticker, issuernm from fisd_fisd.fisd_mergedissue')
mergentHdr['cusip6'] = mergentHdr.hdrcusip.str[:6]
mergentHdr = mergentHdr.rename(columns={'ticker': 'crspTicker'})

### First Route - Link with 6-digit cusip
_cdscrsp1 = pd.merge(redent, mergentHdr, how='left', left_on='entity_cusip', right_on='cusip6')

# store linked results through CUSIP
_cdscrsp_cusip = _cdscrsp1.loc[_cdscrsp1.permno.notna()].copy()
_cdscrsp_cusip['flg'] = 'cusip'

# continue to work with non-linked records
_cdscrsp2 = _cdscrsp1.loc[_cdscrsp1.permno.isna()].copy().drop(columns=['permno','permco','hdrcusip','crspTicker','issuernm','cusip6'])

### Second Route - Link with Ticker
_cdscrsp3 = pd.merge(_cdscrsp2, mergentHdr, how = 'left', left_on = 'ticker', right_on='crspTicker')
_cdscrsp_ticker = _cdscrsp3.loc[_cdscrsp3.permno.notna()].copy()
_cdscrsp_ticker['flg']='ticker'


In [3]:
### Consolidate Output and Company Name Distance Check
cdscrsp = pd.concat([_cdscrsp_cusip, _cdscrsp_ticker], ignore_index=True, axis=0)

# Check similarity ratio of company names
crspNameLst= cdscrsp.issuernm.str.upper().tolist()
redNameLst = cdscrsp.shortname.str.upper().tolist()
len(crspNameLst), len(redNameLst)

nameRatio  = [] # blank list to store fuzzy ratio

for i in range(len(redNameLst)):
    ratio = fuzz.partial_ratio(redNameLst[i], crspNameLst[i])
    nameRatio.append(ratio)
    
cdscrsp['nameRatio']=nameRatio

In [4]:
cdscrsp.sample(5)

,redcode,entity_cusip,ticker,referenceentity,shortname,lei,entity_type,jurisdiction,depthlevel,markitsector,entity_form,companynum_type,companynum,alternativenames,recorddate,validto,validfrom,permno,permco,hdrcusip,crspTicker,issuernm,cusip6,flg,nameRatio
4047,GG8A8N,G0826H,BRL,Baroness Retail Limited,Baroness Retail Ltd,None,Corp,England and Wales,None,Consumer Services,None,English Company Number,4843152,None,None,2020-10-13,None,11415.0,9252.0,06830610,BRL,BARR PHARMACEUTICALS INC,068306,ticker,47
2576,8DCDEH,857689,STN,"STATION CASINOS, INC.",Sta Casinos Inc,None,Corp,Nevada (US),None,Consumer Services,CORP,None,None,None,2008-09-18,2012-01-17,None,79192.0,12158.0,85768910,STN,STATION CASINOS INC,857689,cusip,80
3468,348FFA,317873,FINL,Republic of Finland,Rep Finland,743700M6Y2OQRVSBRD14,Sov,Finland,Low,Government,None,None,None,None,2013-07-08,None,None,34842.0,1733.0,31754010,FINL,FINANCIAL INC,317540,ticker,36
3986,EE475E,E0432C,ALT,ALTADIS SA,ALTADIS SA,959800N8NMPK0SFJGD28,Corp,Spain,Med,Consumer Goods,SA,None,None,None,2006-11-27,None,None,93100.0,53267.0,46429410,ALT,ISHARES TRUST,464294,ticker,40
1269,469FEC,427866,HSY,Hershey Foods Corporation,Hershey Foods Corp,None,Corp,Delaware (US),None,Consumer Goods,CORP,None,None,None,2005-01-26,2005-04-19,None,16600.0,20906.0,42786610,HSY,HERSHEY CO,427866,cusip,90
